In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

plot_dir = '/Users/kevin/Desktop/DRAW/plots/'
data_dir = '/Users/kevin/Desktop/DRAW/data/'

GPU = True

In [2]:
if GPU:
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(423212)
else:
    torch.manual_seed(423212)

## Simple Auto-Encoder

We'll start with the simplest autoencoder: a single, fully-connected layer as the encoder and decoder.

In [3]:
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Linear(input_dim, encoding_dim)
        self.decoder = nn.Linear(encoding_dim, input_dim)

    def forward(self, x):
        encoded = F.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        return decoded

In [4]:
kwargs = {'num_workers': 1, 'pin_memory': True} if GPU else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
                    batch_size=10, shuffle=False, **kwargs)

In [ ]:
input_dim = 784
encoding_dim = 32

model = AutoEncoder(input_dim, encoding_dim)
if GPU:
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters())

In [8]:
def train(epoch, mse=True):
    for batch_idx, (data, _) in enumerate(train_loader):
        if GPU:
            data = data.cuda()
        data = Variable(data.view([-1, 784]))
        optimizer.zero_grad()
        output = model(data)
        if mse:
            loss = F.mse_loss(output, data)
        else:
            loss = F.binary_cross_entropy_with_logits(output, data)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [ ]:
num_epochs = 50
mse = True

for epoch in range(1,  num_epochs + 1):
    train(epoch, mse)

In [ ]:
# encode and decode the test set
data, _ = next(iter(test_loader))
if GPU:
    data = data.cuda()
data = Variable(data.view([-1, 784]), volatile=True)
true_imgs = data
encoded_imgs = F.relu(model.encoder(data))
decoded_imgs = model.decoder(encoded_imgs)

In [ ]:
def to_img(x):
    if GPU:
        x = x.cpu()
    x = x.data.numpy()
    x = x.reshape([-1, 28, 28])
    return x

true_imgs = to_img(true_imgs)
decoded_imgs = to_img(decoded_imgs)

In [ ]:
n = 10

plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(true_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Deep Autoencoder

Let's start stacking up layers like pancakes :)

In [5]:
class DeepAutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(DeepAutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), 
            nn.Linear(64, encoding_dim), 
            nn.ReLU(True),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), 
            nn.Linear(128, input_dim),
            nn.Tanh()
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
input_dim = 784
encoding_dim = 32

model = DeepAutoEncoder(input_dim, encoding_dim)
if GPU:
    model = model.cuda()

optimizer = optim.Adam(model.parameters())

In [9]:
num_epochs = 25
mse = True

for epoch in range(1,  num_epochs + 1):
    train(epoch, mse)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.988188
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.697492
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.590107
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.545805
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.521984
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.535494
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.498544
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.496674
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.489177
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.490673
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.484289
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.488798
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.472341
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.448062
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.461516
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.460681
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.456306
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.460057
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.453341
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.440613
Train Epoch:

Process Process-13:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kev/envs/deepL/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 41, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])


Train Epoch: 13 [0/60000 (0%)]	Loss: 0.397865


Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kev/envs/deepL/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 51, in _pin_memory_loop
    r = in_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 345, in get
    return ForkingPickler.loads(res)
  File "/home/kev/envs/deepL/lib/python3.5/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/usr/lib/python3.5/multiprocessing/

KeyboardInterrupt: 

  File "/home/kev/envs/deepL/lib/python3.5/site-packages/torchvision/datasets/mnist.py", line 72, in __getitem__
    img = Image.fromarray(img.numpy(), mode='L')
  File "/home/kev/envs/deepL/lib/python3.5/site-packages/PIL/Image.py", line 2438, in fromarray
    return frombuffer(mode, size, obj, "raw", rawmode, 0, 1)
  File "/home/kev/envs/deepL/lib/python3.5/site-packages/PIL/Image.py", line 2386, in frombuffer
    core.map_buffer(data, size, decoder_name, None, 0, args)
KeyboardInterrupt


In [ ]:
# encode and decode the test set
data, _ = next(iter(test_loader))
if GPU:
    data = data.cuda()
data = Variable(data.view([-1, 784]), volatile=True)
true_imgs = data
encoded_imgs = F.relu(model.encoder(data))
decoded_imgs = model.decoder(encoded_imgs)

In [ ]:
def to_img(x):
    if GPU:
        x = x.cpu()
    x = x.data.numpy()
    x = x.reshape([-1, 28, 28])
    return x

true_imgs = to_img(true_imgs)
decoded_imgs = to_img(decoded_imgs)

In [ ]:
n = 10

plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(true_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Convolutional Autoencoder

In [ ]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.MaxPool2d(2, stride=1),  # b, 8, 2, 2
            nn.ReLU(True),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
input_dim = 784
encoding_dim = 32

model = ConvAutoEncoder()
if GPU:
    model = model.cuda()

optimizer = optim.Adam(model.parameters())

In [ ]:
def train(epoch, mse=True):
    for batch_idx, (data, _) in enumerate(train_loader):
        if GPU:
            data = data.cuda()
        data = Variable(data)
        optimizer.zero_grad()
        output = model(data)
        if mse:
            loss = F.mse_loss(output, data)
        else:
            loss = F.binary_cross_entropy_with_logits(output, data)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [ ]:
num_epochs = 75
mse = True

for epoch in range(1,  num_epochs + 1):
    train(epoch, mse)